# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'linkedin page', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter page', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [15]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [16]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 4 relevant links


{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'professional profile',
   'url': 'https://www.linkedin.com/in/eddonner/'}]}

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [16]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [17]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
Reachy Mini: The Open Robot for AI Builders
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-OCR
Updated
about 4 hours ago
•
32.9k
•
1.22k
PaddlePaddle/PaddleOCR-VL
Updated
about 23 hours ago
•
6.62k
•
924
Qwen/Qwen3-VL-8B-Instruct
Updated
7 days ago
•
117k
•
272
nanonets/Nanonets-OCR2-3B
Updated
6 days ago
•
16.2k
•
369
Phr00t/Qwen-Image-Edit-Rapid-AIO
Updated
about 8 hours ago
•
388
Browse 1M+ models
Spaces
Running
464
464
veo3.1-fast
🐨
Generate videos from text or images
Running
15.3k
15.3k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
406
406
Sora 2
📉
Gener

In [ ]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [13]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nReachy Mini: The Open Robot for AI Builders\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-OCR\nUpdated\nabout 4 hours ago\n•\n32.9k\n•\n1.22k\nPaddlePaddle/PaddleOCR-VL\nUpdated\nabout 23 hours ago\n•\n6.62k\n•\n924\nQwen/Qwen3-VL-8B-Instruct\nUpdated\n7 days ago\n•\n117k\n•\n272\nnanonets/Nanonets-OCR2-3B\nUpdated\n6 days ago\n•\

In [11]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face: Where AI Meets High-Fives 🤖✋

Welcome to **Hugging Face**, the buzzing community and platform where the world’s machine learning magicians unite, share, and build the future of artificial intelligence — one model, dataset, and quirky AI app at a time!

---

## Who Are We?

Imagine a vibrant hub where over **1 million models**, **250k+ datasets**, and **400k+ applications** are freely explored, updated, and collaborated on daily. That’s Hugging Face: your AI playground, research lab, and creative studio rolled into one.

We’re not just a platform — we’re a *collaboration extravaganza* for ML engineers, data scientists, and AI enthusiasts who want to:

- **Discover** the latest and greatest in models (hello, DeepSeek-OCR and Qwen-Image-Edit-Rapid-AIO!)
- **Contribute** data and datasets that power the next epoch of AI brilliance
- **Build & Share** in our lively “Spaces” — mini universes where you generate videos from text, create applications with Vibe Coding, and even animate images faster than you can say “machine learning” (well, maybe a bit slower).

---

## Why Join the Hugging Face Party?

**Community-Vibes:** Engage with a fast-growing, friendly bunch of AI dreamers and doers. We’re serious about open and ethical AI, but we don’t take ourselves too seriously — after all, there’s a 🐨 koala mascot and robot named Reachy Mini hanging out.

**Open Source, Open Minds:** HF’s open-source stack is your secret weapon for speeding through ML projects. Whether your passion is text, image, video, audio, or even 3D, there's a tool or model waiting for you.

**Build Your AI Résumé:** Share your projects, see what others are cooking, and build your profile. It's like LinkedIn… but way cooler because it’s full of AI rockstars.

**Enterprise & Compute Power:** Got a team or large-scale AI ambitions? Our paid Compute and Enterprise solutions have your back — with top-tier tech to accelerate your AI goals.

---

## For Prospective Customers & Investors

Looking to tap into the most **dynamic AI ecosystem** on the planet? Hugging Face is:

- The **go-to hub** for deploying cutting-edge open-source AI tech.
- A thriving community of **innovators fueling AI breakthroughs** in speech, vision, and text.
- Your partner to **scale AI solutions efficiently** with flexible compute and enterprise-grade tools.

Join the companies and researchers who trust Hugging Face to power their AI projects swiftly and ethically.

---

## For Future Hugging Face Team Members 🧑‍💻

Dream of building the future where AI actually *understands* humans (better than your in-laws, hopefully)? Here at Hugging Face:

- You’re part of a **passionate, inclusive community** that’s rewriting the AI playbook.
- The culture thrives on **collaboration, creativity**, and **freedom to innovate** in an open-source environment.
- Work on **cutting-edge tech** like HuggingChat Omni (a chatty AI buddy) and Reachy Mini (tiny robot, big personality).
- Your code never sleeps — it’s part of a global ecosystem helping thousands of projects evolve in real-time.

If you want your career to be as dynamic as a rapidly updating ML model, **Hugging Face wants you!**

---

## Fun Facts to Impress Your AI-Fan Friends:

- You can chat with AI instantly via our new **HuggingChat Omni** — your next favorite digital pal.
- We host a gazillion models and growing — from OCR to image editing to video generation. Yes, AI content creators, this is your candy store.
- Our colors? Sunshine yellow (#FFD21E), fiery orange (#FF9D00), and chill slate grey (#6B7280) — because AI deserves style too.

---

## Ready to Jump In?

Sign up, explore, innovate, and hug your way into the future of AI at [huggingface.co](https://huggingface.co).

Whether you want to build the next viral ML model, invest in groundbreaking AI tech, or join a company where nerdy humor and cutting-edge machine learning collide — Hugging Face has a spot waiting for you.

*Come for the models. Stay for the community. Leave with a smarter, friendlier AI world.* 

---

# Hugging Face™  
The AI community that’s hugging the future — one neural net at a time! 🤗

In [25]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

# Welcome to Hugging Face  
*The AI community building the future — and having a bit of fun doing it!*

---

## Who We Are  
Hugging Face is not just a company; we're the cool hangout spot for machine learning enthusiasts, scientists, and AI fans worldwide. Founded in 2016 and headquartered in Paris, we proudly wear the badge of “AI community builders” with 51-200 passionate employees onboard — a lively bunch pushing the boundaries of NLP, deep learning, and much more.

---

## What We Do  
Think of us as the GitHub for machine learning — the go-to hub where anyone, from AI whizzes to curious newcomers, can find:

- **1 million+ models:** From natural language processing that actually *talks* your language, to obscure deep learning wonders.
- **250,000+ datasets:** Because good data = good AI vibes.
- **400,000+ community applications:** Where ideas get real and sometimes hilarious.
- **Spaces:** Our playgrounds where ML magic happens live.

And now, try **HuggingChat Omni** — talk to AI like it’s your new best friend.  
Or meet **Reachy Mini**, your open-source robot buddy for all AI builder adventures.

---

## Why Hugging Face?  
- **Openness & Collaboration:** Share your latest ML masterpiece or remix others’ work. It’s all about growing our AI family together.  
- **Ethical AI:** Building trustworthy AI systems that everyone can trust. No black boxes here!  
- **Multimodal Magic:** Text, images, videos, audio — even 3D. If it's data, we embrace it.  
- **Speedy Innovation:** With open-source tools and paid compute options, you’ll be faster than ever hitting your AI goals.

---

## Culture — Serious Work, Not-So-Serious People  
- **Inclusivity:** Everyone’s welcome, whether you write code, datasets, poems, or just good vibes.  
- **Community-First:** Growth fueled by support, collaboration, and an open door — or should we say open repo?  
- **Creative playground:** Work on bleeding-edge AI, rocket science adjacent, with a sprinkle of fun.  
- **Transparency:** No AI mysteries here. We're all about open data—because sharing is caring.

---

## Join Us!  
Looking to code your future with us? Whether you're a research lead, engineer, or AI curious cat, our careers page is constantly buzzing with new opportunities in the AI revolution.

**Bonus:** Perks include working with some of the brightest minds, in a global community that’s changing the world one model at a time.

---

## Our Customers: Big, Small & Curious  
From startups, big enterprises, to researchers and hobbyists, we serve anyone interested in democratizing AI. If AI is the future, consider us your AI home base.

---

## Fun Fact Corner  
- Our logo? It's literally a smiling face giving you a *hug*. Because who says AI can't be warm and fuzzy?  
- Got 10+ years of AI experience? We're practically neighbors — our team is stacked with veteran researchers and builders pushing boundaries daily.

---

## Ready to Hug the Future?  
Come build, break, remake, and celebrate AI with us at [huggingface.co](https://huggingface.co) — where machines learn, people connect, and the future is collaboratively coded.

---

### Hugging Face  
*Not your average AI company... because AI needs a little love too.*  

---

# Join us, and let's build the AI future — one model, one dataset, one smile at a time. 🤗

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>